In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
from macd import compute_ema
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta
from stocktrends import Renko

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [4]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [5]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [7]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [8]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865


In [9]:
df = infy_df.copy()

In [10]:
# ! pip install stocktrends

In [11]:
def convert_to_renko(df,brick_size):
    df = df.copy()
    df.reset_index(inplace=True)
    renko_df = Renko(df)
    renko_df.brick_size = brick_size
    renko_df = renko_df.get_ohlc_data()
    return renko_df


In [12]:
renko_df = convert_to_renko(df,7.6091)

In [13]:
renko_df

,date,open,high,low,close,uptrend
0,2024-06-03 09:15:00+05:30,1415.2926,1422.9017,1415.2926,1422.9017,True
1,2024-06-03 10:55:00+05:30,1415.2926,1415.2926,1407.6835,1407.6835,False
2,2024-06-04 09:15:00+05:30,1407.6835,1407.6835,1400.0744,1400.0744,False
3,2024-06-04 09:20:00+05:30,1400.0744,1400.0744,1392.4653,1392.4653,False
4,2024-06-04 10:20:00+05:30,1400.0744,1407.6835,1400.0744,1407.6835,True
5,2024-06-04 11:55:00+05:30,1400.0744,1400.0744,1392.4653,1392.4653,False
6,2024-06-04 12:10:00+05:30,1392.4653,1392.4653,1384.8562,1384.8562,False
7,2024-06-04 12:20:00+05:30,1384.8562,1384.8562,1377.2471,1377.2471,False
8,2024-06-04 12:25:00+05:30,1377.2471,1377.2471,1369.638,1369.638,False
9,2024-06-04 12:45:00+05:30,1377.2471,1384.8562,1377.2471,1384.8562,True
